In [1]:
import ee
import geemap

from src.helpers.classifier import SmileRandomForest

In [2]:
model = SmileRandomForest.load("projects/nb-lidar/assets/dsm_rf_model")
model

In [3]:
model.model

In [9]:
from src.helpers.image_processing import process_and_stack_images

aoi = ee.FeatureCollection("projects/nb-lidar/assets/aoi_nb_south").geometry()

stack = process_and_stack_images(aoi, "dsm")

In [10]:
# predict
prediction = model.predict(stack)

In [11]:
Map = geemap.Map()

Map.addLayer(prediction, {"min": 1, "max": 8})

Map.centerObject(aoi, 10)

Map

Map(center=[45.74809742262684, -65.99999969409251], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
AOI_ID = "projects/nb-lidar/assets/aoi_nb_south"
BUCKET = "eerfpl-exports"
terrain_type = "dsm"

name = f"{terrain_type}_prdiction"
file_name_prefix = f"nb-lidar/{name}-2/{name}-"

cloud_task = ee.batch.Export.image.toCloudStorage(
    image=prediction,
    description="",
    bucket=BUCKET,
    fileNamePrefix=file_name_prefix,
    region=aoi,
    maxPixels=1e13,
    fileDimensions=[2048, 2048],
    skipEmptyTiles=True,
    scale=10,
)

cloud_task.start()